In [9]:
import pandas as pd
from casauth import CASAuth
import swat
import os

path = os.getenv('CAS_CREDENTIALS')
pem_file = os.getenv('CAS_CLIENT_SSL_CA_LIST')

conn = CASAuth(path, ssl_ca_list = pem_file)
conn.about()['About']['Viya Version']

The access token may have expired - attempting to refresh the token
The access token was stored for you in the access_token.txt file. The access token expires in 24 hours.
CAS Connection created
NOTE: Grid node action status report: 5 nodes, 9 total actions executed.


'Stable 2023.05'

# Create data

In [85]:
df = pd.DataFrame({'population': [59000000, 65000000, 434000,
                                  434000, 434000, 337000, 11300,
                                  11300, 11300, 65000000],
                   'GDP': [1937894, 2583560 , 12011, 4520, 12128,
                           17036, 182, 38, 311, 2583560],
                   'alpha-2': ["IT", "FR", "MT", "MV", "BN",
                               "IS", "NR", "TV", "AI","TEST"]},
                  index=["Italy", "France", "Malta",
                         "Maldives", "Brunei", "Iceland",
                         "Nauru", "Tuvalu", "Anguilla","TestCtry"])
df

castbl = conn.upload_file(df.reset_index(), casout={'name':'tbl', 'replace':True})

NOTE: Cloud Analytic Services made the uploaded file available as table TBL in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table TBL has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


In [86]:
df.head(10)

,population,GDP,alpha-2
Italy,59000000,1937894,IT
France,65000000,2583560,FR
Malta,434000,12011,MT
Maldives,434000,4520,MV
Brunei,434000,12128,BN
Iceland,337000,17036,IS
Nauru,11300,182,NR
Tuvalu,11300,38,TV
Anguilla,11300,311,AI
TestCtry,65000000,2583560,TEST


In [87]:
castbl.head(10)

,index,population,GDP,alpha-2
0,Italy,59000000.0,1937894.0,IT
1,France,65000000.0,2583560.0,FR
2,Malta,434000.0,12011.0,MT
3,Maldives,434000.0,4520.0,MV
4,Brunei,434000.0,12128.0,BN
5,Iceland,337000.0,17036.0,IS
6,Nauru,11300.0,182.0,NR
7,Tuvalu,11300.0,38.0,TV
8,Anguilla,11300.0,311.0,AI
9,TestCtry,65000000.0,2583560.0,TEST


pandas

In [96]:
df.nlargest(2, ['population'])

,population,GDP,alpha-2
France,65000000,2583560,FR
TestCtry,65000000,2583560,TEST


In [97]:
df.nlargest(6, ['population'])

,population,GDP,alpha-2
France,65000000,2583560,FR
TestCtry,65000000,2583560,TEST
Italy,59000000,1937894,IT
Malta,434000,12011,MT
Maldives,434000,4520,MV
Brunei,434000,12128,BN


In [98]:
df.nlargest(6, ['population','GDP'])

,population,GDP,alpha-2
France,65000000,2583560,FR
TestCtry,65000000,2583560,TEST
Italy,59000000,1937894,IT
Brunei,434000,12128,BN
Malta,434000,12011,MT
Maldives,434000,4520,MV


swat

In [100]:
topkdf = castbl.topk(topk=2, bottomk=0, inputs = ['population'], raw = True)['Topk']
topkdf

,Column,NumVar,Rank
0,population,65000000.0,1
1,population,59000000.0,2


In [101]:
filterTable = conn.upload_frame(topkdf, casout={'name':'TEMP_TBL', 'caslib':'casuser','replace':True})
filterTable.head(10)

NOTE: Cloud Analytic Services made the uploaded file available as table TEMP_TBL in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table TEMP_TBL has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


,Column,NumVar,Rank
0,population,65000000.0,1.0
1,population,59000000.0,2.0


In [80]:
filterTable.alterTable(columns = [{'name':'NumVar', 'rename':'Population'}])
filterTable.head()

,Column,Population,Rank
0,population,65000000.0,1.0
1,population,59000000.0,2.0
2,population,434000.0,3.0


In [81]:
filterTable.vars = [{'name':'Population'}]
filterTable.head()

,Population
0,65000000.0
1,59000000.0
2,434000.0


In [90]:
filterTable

CASTable('TEMP_TBL', caslib='CASUSER(Peter.Styliadis@sas.com)', vars=[dict(name='Population')])

In [88]:
castbl

CASTable('TBL', caslib='CASUSER(Peter.Styliadis@sas.com)')

In [94]:
conn.table.fetch(table = {'name':'TBL', 
                          'caslib':'casuser', 
                          'whereTable': {'name':'TEMP_TBL',
                                         'caslib':'casuser',
                                         'vars':[{'name':'Population'}]
                                        } 
                         })

,index,population,GDP,alpha-2
0,Italy,59000000.0,1937894.0,IT
1,France,65000000.0,2583560.0,FR
2,Malta,434000.0,12011.0,MT
3,Maldives,434000.0,4520.0,MV
4,Brunei,434000.0,12128.0,BN
5,TestCtry,65000000.0,2583560.0,TEST
